In [51]:
import geopandas as gpd
import mapclassify
import matplotlib.pyplot as plt


In [52]:
url = "https://github.com/DACSS-CSSmeths/Spatial-Exploring/raw/refs/heads/main/maps/countriesCIA.gpkg"
gdf = gpd.read_file(url)

In [53]:
gdf.head()


,COUNTRY,name,region,obesityAdults_rate,TobaccoUse_perc,Alcohol_LitersPerCap,tobacco_code,tobacco_levels,tobacco_custom,alcohol_code,alcohol_levels,geometry
0,Afghanistan,Afghanistan,South Asia,5.5,23.3,0.01,2,3.average,3.(15-30],0,1.very low,"MULTIPOLYGON (((61.27656 35.60725, 61.29638 35..."
1,Algeria,Algeria,Africa,27.4,21.0,0.59,2,3.average,3.(15-30],0,1.very low,"MULTIPOLYGON (((-5.15213 30.18047, -5.13917 30..."
2,Azerbaijan,Azerbaijan,Middle East,19.9,24.0,1.38,2,3.average,3.(15-30],0,1.very low,"MULTIPOLYGON (((46.54037 38.87559, 46.49554 38..."
3,Albania,Albania,Europe,21.7,22.4,4.40,2,3.average,3.(15-30],2,3.average,"MULTIPOLYGON (((20.79192 40.43154, 20.78722 40..."
4,Armenia,Armenia,Middle East,20.2,25.5,3.77,2,3.average,3.(15-30],1,2.low,"MULTIPOLYGON (((46.54037 38.87559, 46.51639 38..."


In [54]:
print(gdf.columns)

Index(['COUNTRY', 'name', 'region', 'obesityAdults_rate', 'TobaccoUse_perc',
       'Alcohol_LitersPerCap', 'tobacco_code', 'tobacco_levels',
       'tobacco_custom', 'alcohol_code', 'alcohol_levels', 'geometry'],
      dtype='object')


In [55]:
gdf = gdf.rename(columns={
    "obesityAdults_rate": "obesity",
    "TobaccoUse_perc": "tobacco",
    "Alcohol_LitersPerCap": "alcohol"
})

# Just to confirm the rename worked:
print(gdf.columns)

Index(['COUNTRY', 'name', 'region', 'obesity', 'tobacco', 'alcohol',
       'tobacco_code', 'tobacco_levels', 'tobacco_custom', 'alcohol_code',
       'alcohol_levels', 'geometry'],
      dtype='object')


In [56]:
import mapclassify

# 1) Determine the best scheme for 'obesity'
schemes = [
    mapclassify.FisherJenks,
    mapclassify.NaturalBreaks,
    mapclassify.EqualInterval,
    mapclassify.Quantiles
]

best_scheme = None
lowest_adcm = float('inf')
best_classifier = None

for scheme in schemes:
    classifier = scheme(gdf["obesity"].dropna(), k=5)
    adcm = classifier.adcm  # average distance from class means
    if adcm < lowest_adcm:
        lowest_adcm = adcm
        best_scheme = scheme.__name__
        best_classifier = classifier

print(f"Best scheme for 'obesity' is {best_scheme} with ADCM = {lowest_adcm}")

# 2) Use the best scheme to bin all three columns
def bin_variable(df, col, scheme_class):
    classifier = scheme_class(df[col].dropna(), k=5)
    df[col + "_bin"] = classifier.find_bin(df[col])
    return df

# Decide which scheme class to use, based on the best_scheme string:
if best_scheme == "FisherJenks":
    chosen_scheme = mapclassify.FisherJenks
elif best_scheme == "NaturalBreaks":
    chosen_scheme = mapclassify.NaturalBreaks
elif best_scheme == "EqualInterval":
    chosen_scheme = mapclassify.EqualInterval
elif best_scheme == "Quantiles":
    chosen_scheme = mapclassify.Quantiles

# Bin obesity, alcohol, and tobacco
gdf = bin_variable(gdf, "obesity", chosen_scheme)
gdf = bin_variable(gdf, "alcohol", chosen_scheme)
gdf = bin_variable(gdf, "tobacco", chosen_scheme)

# 3) Identify the best countries (lowest bin == 1 for all three)
best_countries = gdf[
    (gdf["obesity_bin"] == 0) &
    (gdf["alcohol_bin"] == 0) &
    (gdf["tobacco_bin"] == 0)
]
print("Best countries in obesity, alcohol, and tobacco:")
best_countries[["COUNTRY", "obesity", "alcohol", "tobacco"]]

# 4) Save to a new geopackage
output_file = "countriesCIA_binned.gpkg"
gdf.to_file(output_file, driver="GPKG")
print(f"Saved updated geopackage: {output_file}")

Best scheme for 'obesity' is FisherJenks with ADCM = 283.7999999999999
Best countries in obesity, alcohol, and tobacco:
Saved updated geopackage: countriesCIA_binned.gpkg
